### Modules used in this notebook
pypsa, plotnine

# E.002 Extending this model to include two countries
In this notebook you will create a simple power system model simulating a system with two regions connected together with specific month of the year using wind based on atmospheric reanalysis data. The model will be optimised to define the optimal capacity of batteries needed to balance the system.

# Step 01: Setting up the environment
This part of the notebook install the needed Python modules and the solver ([cbc](https://github.com/coin-or/Cbc), a widely-used open source solver).
The installed modules are:
  - PyPSA: [an open-source power system model](https://pypsa.org/)

In [ ]:
!pip install plotnine
!pip install pypsa # power system model
!apt-get install -y -qq coinor-cbc # open source solver

# Step 02: Read wind, demand data
Here the download a set of Parquet files containing the electricity demand, wind and solar profiles based on [this dataset](https://researchdata.reading.ac.uk/272/) developed by Bloomfield et al., University of Reading.

The Parquet files provide a data frame with the hourly values for all the European countries for the years 2000-2019 (the original dataset goes back to 1979 but we have kept only the last 20 years to reduce the amount of storage needed to run this notebook).

In [ ]:
import pandas as pd
dem = pd.read_parquet('../data/ERA5_full_demand_all_countries_2000_2019_hourly.parquet')
# pv  = pd.read_parquet('ERA5_solar_power_capacity_factor_all_countries_2000_2019_hourly.parquet')
win = pd.read_parquet('../data/ERA5_wind_power_capacity_factor_all_countries_2000_2019_inclusive.parquet')
dem.head()

# Step 03: selecting year, month, and the two countries
In this example we simulate a power system model consisting of two regions, each one with a wind power farm and a lithium-ion battery, connected together with an interconnector. We specify the electricity demand and the RES profiles (capacity factors) for a specific month.

In [ ]:
selected_year = 2012 # 2000-2019
selected_month = 2 # 1 = January, 12 = December
country01 = 'Spain'
country02 = 'Denmark'

In [ ]:
# Select demand  and wind profiles
selected_dem_01 = dem.loc[(pd.DatetimeIndex(dem['datetime']).year == selected_year) & (pd.DatetimeIndex(dem['datetime']).month == selected_month)][country01]
selected_win_01 = win.loc[(pd.DatetimeIndex(win['datetime']).year == selected_year) & (pd.DatetimeIndex(win['datetime']).month == selected_month)][country01]

selected_dem_02 = dem.loc[(pd.DatetimeIndex(dem['datetime']).year == selected_year) & (pd.DatetimeIndex(dem['datetime']).month == selected_month)][country02]
selected_win_02 = win.loc[(pd.DatetimeIndex(win['datetime']).year == selected_year) & (pd.DatetimeIndex(win['datetime']).month == selected_month)][country02]

# Step 04: Building the power system model
We create here a single-bus PyPSA network with:
  - `Wind`: A wind generator with the cost of 100 000 EUR per MW
  - `Battery`: a battery with 3 hours of storage with the cost of 30 000 EUR per MW. The efficiency is 90%.

The system has an electricity demand defined by the load `L1`

In [ ]:
import pypsa, logging
pypsa.pf.logger.setLevel(logging.WARNING)

network = pypsa.Network(snapshots = selected_dem_01.index)
network.add("Bus", "B1")
network.add("Bus", "B2")

network.add("Generator", "Wind01",
            bus="B1",
            capital_cost = 100_000,
            p_max_pu = selected_win_01,
            p_nom_extendable = True,
            control="PQ")

network.add("Generator", "Wind02",
            bus="B2",
            capital_cost = 100_000,
            p_max_pu = selected_win_02,
            p_nom_extendable = True,
            control="PQ")


network.add("StorageUnit", "Battery01",
            bus="B1",
            capital_cost = 30_000,
            efficiency_store = 0.9,
            efficiency_dispatch = 0.9,
            max_hours = 3,
            p_nom_extendable = True
            )

network.add("StorageUnit", "Battery02",
            bus="B2",
            capital_cost = 30_000,
            efficiency_store = 0.9,
            efficiency_dispatch = 0.9,
            max_hours = 3,
            p_nom_extendable = True
            )

network.add('Link', 'ITC01-02',
            bus0 = 'B1',
            bus1 = 'B2',
            p_min_pu = -1,
            marginal_cost = 0.1,
            p_nom = 8000 )

network.add("Load", "L1",
            bus="B1",
            p_set=selected_dem_01*1e3)
network.add("Load", "L2",
            bus="B2",
            p_set=selected_dem_02*1e3)


# Step 05: Running the model
Here we solve the linear optimal power flow defined in the PyPSA network created in the step before. We print the optimal capacities (in MW) of wind, solar and batteries found by the solver.

In [ ]:
network.optimize(solver_name = 'cbc')

In [ ]:
network.statistics.expanded_capacity()

# Step 06: Plotting the dispatching
To plot the generation of the three assets modeled in this example we need to insert the data contained in the `network` object into a Pandas data frame. This is eventually used for a plot made with [plotnine](https://plotnine.readthedocs.io/en/stable/)

In [ ]:
gen_list = network.generators[['bus']].reset_index()
gen_list['type'] = gen_list['Generator'].str.split('_', expand = True).iloc[:,0]
gen = (pd.merge(
    network.generators_t.p
    .unstack()
    .reset_index(), gen_list)
       .groupby(['type', 'bus', 'snapshot'])
       .sum()
       .reset_index()
      ).rename(columns = {0: 'prod'})

sto_list = network.storage_units[['bus']].reset_index()
sto_list['type'] = sto_list['StorageUnit'].str.split('_', expand = True).iloc[:,0]
sto = (pd.merge(
    network.storage_units_t.p
    .unstack()
    .reset_index(), sto_list)
       .groupby(['type', 'bus', 'snapshot'])
       .sum().reset_index()
      ).rename(columns = {0: 'prod'})

df = pd.concat([gen, sto])
df.head()

In [ ]:
import plotnine
plotnine.options.figure_size = (12, 4)
# First country
(
    plotnine.ggplot(pd.DataFrame(network.loads_t.p_set.L1).reset_index(), plotnine.aes(x='snapshot', y='L1')) +
    plotnine.geom_area(plotnine.aes(x = 'snapshot', y = 'prod', fill = 'type'), data= df.query("bus == 'B1'")) +
    plotnine.theme_bw() +
    plotnine.geom_line()
)


In [ ]:
# Second country
(
    plotnine.ggplot(pd.DataFrame(network.loads_t.p_set.L2).reset_index(), plotnine.aes(x='snapshot', y='L2')) +
    plotnine.geom_area(plotnine.aes(x = 'snapshot', y = 'prod', fill = 'type'), data= df.query("bus == 'B2'")) +
    plotnine.theme_bw() +
    plotnine.geom_line()
)

In [ ]:
# ITC
(
    plotnine.ggplot(pd.DataFrame(network.links_t.p0).reset_index(), plotnine.aes(x='snapshot', y='ITC01-02')) +
    plotnine.geom_hline(yintercept=0, linetype = 'dashed') +
    plotnine.geom_line() +
    plotnine.theme_bw()
)
